In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import csv
import requests
from io import StringIO
from scipy.stats import norm
from scipy.stats import t

In [ ]:
class ADbib:
    @staticmethod
    def media_aritmetica(dados):
      if not dados:
        raise ValueError("Lista de dados vazia.")
      soma = sum(dados)
      n = len(dados)

      return round(soma/n, 3)
    
    @staticmethod
    def media_ponderada(dados, pesos):
      if len(dados) != len(pesos):
        raise ValueError("Listas de dados e pesos com tamanhos diferentes.")
      numerador = sum([dado * peso for dado, peso in zip(dados, pesos)])
      denominador = sum(pesos)
      if denominador == 0:
        raise ZeroDivisionError("Soma dos pesos é zero.")

      return round(numerador/denominador,3)
    
    @staticmethod
    def media_geometrica(dados):
      if not dados:
        raise ValueError("Lista de dados vazia.")
      if any(valor <= 0 for valor in dados):
        raise ValueError("Dados devem ser maiores que zero.")
      produto = 1
      for valor in dados:
        produto *= valor
      n = len(dados)

      return round(produto ** (1/n),3)
    
    @staticmethod
    def media_harmonica(dados):
      if not dados:
        raise ValueError("Lista de dados vazia.")
      if any(valor == 0 for valor in dados):
        raise ValueError("Dados devem ser diferentes de zero.")

      inversos = [1 / valor for valor in dados]
      n = len(dados)

      return int(n/sum(inversos))
    
    @staticmethod
    def media_taxas(denominadores, taxas):
        n = len(taxas)

        if all(valor > 0 for valor in denominadores):
            if not ADbib.todos_iguais(taxas) and not ADbib.todos_iguais(denominadores):
                return round((ADbib.media_aritmetica(taxas)/ADbib.media_aritmetica(denominadores))*100,3)

            elif ADbib.todos_iguais(denominadores) != 0:
                return round((1/(n * denominadores[0])*sum(taxas))*100,3)

            elif sum(taxas) != 0 and sum(taxas) == taxas[0]*n:
                return round((n/ADbib.media_harmonica(taxas))*100, 3)

            else:
                return None

    @staticmethod
    def mediana(dados):
        dados_ordenados = ADbib.fazer_rol(dados)
        n = len(dados_ordenados)
        if n % 2 == 1:
            return dados_ordenados[n // 2]
        else:
            return (dados_ordenados[n // 2 - 1] + dados_ordenados[n // 2]) / 2
    
    @staticmethod
    def moda(dados):
        if not dados:
            raise ValueError("Lista de dados vazia")
        contador = {}
        for valor in dados:
            contador[valor] = contador.get(valor, 0) + 1
        moda = max(contador, key=contador.get)
        return moda
    
    @staticmethod
    def distribuicao_frequencia_com_histograma(dados):
        distribuicao = {}
        for valor in dados:
            distribuicao[valor] = distribuicao.get(valor, 0) + 1
        
        valores = list(distribuicao.keys())
        frequencias = list(distribuicao.values())
        
        plt.bar(valores, frequencias)
        plt.xlabel('Valores')
        plt.ylabel('Frequência')
        plt.title('Histograma da Distribuição de Frequências')
        plt.show()

    @staticmethod
    def desvio_padrao(dados):
      if not dados:
        raise ValueError("Lista de dados vazia.")
      
      media = (ADbib.variancia_amostral(dados))**(1/2)
      return round(media,3)

    @staticmethod
    def amplitude(dados):
        return max(dados) - min(dados)
    
    @staticmethod
    def variancia_amostral(dados):
        media = ADbib.media_aritmetica(dados)
        total = 0
        for i in dados:
            total += (i - media)**2
    
        return total/(len(dados)-1)
    
    
    @staticmethod
    def coeficiente_variacao(dados):
        desvio_padrao = ADbib.desvio_padrao(dados)
        media = ADbib.media_aritmetica(dados)
        return round((desvio_padrao/media)*100,3)

    @staticmethod
    def todos_iguais(dados):
      if not dados:
        return True
      
      valor_base = dados[0]

      for valor in dados[1:]:
        if valor != valor_base:
          return False
      return True

    @staticmethod
    def merge_sort(arr):
        if len(arr) > 1:
            mid = len(arr) // 2
            left_half = arr[:mid]
            right_half = arr[mid:]

            ADbib.merge_sort(left_half)
            ADbib.merge_sort(right_half)

            i = j = k = 0

            while i < len(left_half) and j < len(right_half):
                if left_half[i] < right_half[j]:
                    arr[k] = left_half[i]
                    i += 1
                else:
                    arr[k] = right_half[j]
                    j += 1
                k += 1

            # Verifica se há elementos restantes nas duas metades
            while i < len(left_half):
                arr[k] = left_half[i]
                i += 1
                k += 1

            while j < len(right_half):
                arr[k] = right_half[j]
                j += 1
                k += 1

    @staticmethod
    def fazer_rol(dados):
        ADbib.merge_sort(dados)
        return dados

    @staticmethod
    def quartis(dados):
        dados_ordenados = ADbib.fazer_rol(dados)
        n = len(dados_ordenados)
        q1_index = int(n * 0.25)
        q2_index = int(n * 0.5)
        q3_index = int(n * 0.75)
        q1 = dados_ordenados[q1_index]
        q2 = dados_ordenados[q2_index]
        q3 = dados_ordenados[q3_index]
        return q1, q2, q3
    
    @staticmethod
    def amplitude_interquartil(dados):
        q1, _, q3 = ADbib.quartis(dados)
        return q3 - q1
    
    @staticmethod
    def diagrama_caixa(dados):
        plt.boxplot(dados)
        plt.title('Diagrama de Caixa')
        plt.show()
    
    @staticmethod
    def z(nivel_confianca):
        alpha = 1 - nivel_confianca / 100
        z = norm.ppf(1 - alpha / 2)
        return round(z,3)

    @staticmethod    
    def t_student(graus_liberdade, nivel_confianca):
        alpha = 1 - nivel_confianca / 100
        return t.ppf(1 - alpha / 2, graus_liberdade)

    @staticmethod    
    def intervalo_confianca(dados,nivel):
      n = len(dados)
      media = ADbib.media_aritmetica(dados)
      desvio_padrao = ADbib.desvio_padrao(dados)
    
      if n > 30:
        coeficiente = ADbib.z(nivel)
    
        limite_inferior = media - (desvio_padrao * coeficiente/(n**(1/2)))
        limite_superior = media + (desvio_padrao * coeficiente/(n**(1/2)))
    
        return (round(limite_inferior,3),round(limite_superior,3))
    
      else:
        distancia_t = ADbib.t_student(n-1,nivel)
    
        limite_inferior = media - (desvio_padrao * distancia_t/(n**(1/2)))
        limite_superior = media + (desvio_padrao * distancia_t/(n**(1/2)))
    
        return (round(limite_inferior,3),round(limite_superior,3))

    @staticmethod
    def teste_media_zero(entrada_a,entrada_b,nivel_confianca):
        n_a = len(entrada_a)
        n_b = len(entrada_b)
    
        media_a = round(ADbib.media_aritmetica(entrada_a),3)
        media_b = round(ADbib.media_aritmetica(entrada_b),3)
    
        desvio_a = round(ADbib.desvio_padrao(entrada_a),3)
        desvio_b = round(ADbib.desvio_padrao(entrada_b),3)
    
        S = ((desvio_a**2/n_a)+(desvio_b**2/n_b))**(1/2)
    
        numerador = ((desvio_a**2/n_a)+(desvio_b**2/n_b))**2
        denominador = (1/(n_a-1)*(desvio_a**2/n_a)**2)+(1/(n_b-1)*(desvio_b**2/n_b)**2)
    
        v = (round(numerador,3)/round(denominador,3)) - 2
    
        limite_inferior = (media_a - media_b) - (S * ADbib.t_student(v,nivel_confianca))
        limite_superior = (media_a - media_b) + (S * ADbib.t_student(v,nivel_confianca))
    
        return [limite_inferior,limite_superior]

In [ ]:
'''MÉTODO PARA ESCOLHER QUAL A MÉDIA A SER USADA:

    @staticmethod
    def selecionar_media(dados, pesos=None, taxas=None):
      if all(valor > 0 for valor in dados):
        #Se todos os valores forem positivos:
          if len(dados) == 2:
            #Se forem apenas 2 valores, usar a média aritmética
            return "Aritmética", ADbib.media_aritmetica(dados)
          elif ADbib.todos_iguais(dados) and pesos is None:
            #Se todos os valores forem iguais, usar a média aritmética
            return "Aritmética", ADbib.media_aritmetica(dados)
          else:
              if pesos is None:
                #Se não houver pesos, usar a média aritmética ou geométrica
                if ADbib.desvio_padrao(dados) == 0:
                  return "Geométrica", ADbib.media_geometrica(dados)
                else:
                  return "Aritmética", ADbib.media_aritmetica(dados)
              else:
                #Se houver pesos, usar a média ponderada ou media de taxas
                  media = ADbib.media_taxas(pesos, dados)
                  if media is None:
                      return "Ponderada", ADbib.media_ponderada(dados, pesos)
                  else:
                      return "Taxas", media
      else:
        #Se houver pelo menos um valor não positivo:
          if all(valor for valor in dados):
            # Se todos os valores forem zero ou positivos:
              if pesos is None:
                #Se não houver pesos, usar a média aritmética
                return "Aritmética", ADbib.media_aritmetica(dados)
              else:
                # Se houver pesos, usar a média ponderada
                return "Ponderada", ADbib.media_ponderada(dados, pesos)
          else:
            #Se houver pelo menos um valor negativo:
              return "Harmônica", ADbib.media_harmonica(dados)
'''